In [1]:
import pandas as pd

# Charger le DataFrame
df1 = pd.read_csv("df_part1.csv", index_col=0)

# Vérifier que la colonne "merged" existe
if "merged" in df1.columns:
    # Calculer le nombre total de caractères
    total_chars = df1["merged"].astype(str).apply(len).sum()
    print(f"Nombre total de caractères dans la colonne 'merged' : {total_chars}")
else:
    print("❌ La colonne 'merged' n'existe pas dans df_part1.csv")

Nombre total de caractères dans la colonne 'merged' : 11569457


In [2]:
import pandas as pd
from langdetect import detect

# Charger le DataFrame
df1 = pd.read_csv("df_part1.csv", index_col=0)

# Vérifier que la colonne "merged" existe
if "merged" in df1.columns:
    # Détection de la langue (avec gestion des erreurs)
    def detect_lang(text):
        try:
            return detect(str(text))  # Détection de la langue
        except:
            return "unknown"  # Si erreur, on met "unknown"

    # Ajouter une colonne "lang" avec la langue détectée
    df1["lang"] = df1["merged"].apply(detect_lang)

    # Filtrer les textes dont la langue n'est pas le français
    df_non_fr = df1[df1["lang"] != "fr"]

    # Calculer le nombre total de caractères
    total_chars_non_fr = df_non_fr["merged"].astype(str).apply(len).sum()

    print(f"Nombre total de caractères dans 'merged' (langues ≠ fr) : {total_chars_non_fr}")
else:
    print("❌ La colonne 'merged' n'existe pas dans df_part1.csv")


Nombre total de caractères dans 'merged' (langues ≠ fr) : 1238598


In [3]:
import pandas as pd

# Charger le DataFrame
df1 = pd.read_csv("df_part1.csv", index_col=0)

# Vérifier que la colonne "merged" existe
if "merged" in df1.columns:
    # Concaténer tous les textes en une seule chaîne
    all_text = "".join(df1["merged"].dropna().astype(str))

    # Extraire les caractères uniques
    unique_chars = sorted(set(all_text))  # Trie pour une meilleure lisibilité

    print(f"Caractères uniques trouvés ({len(unique_chars)} au total) :")
    print("".join(unique_chars))  # Affiche sous forme de chaîne
else:
    print("❌ La colonne 'merged' n'existe pas dans df_part1.csv")

Caractères uniques trouvés (33 au total) :
 DINPRSabcdefghijklmnopqrstuvwxyz


Reperer les doublons de descriptions avant

In [4]:
import pandas as pd

# Charger le DataFrame
df_clean = pd.read_csv("df_part1.csv")

# Vérifier que la colonne "merged" existe
if "merged" in df_clean.columns:
    # Fonction pour supprimer les doublons de mots dans une cellule
    def remove_duplicate_words(text):
        words = str(text).split()  # Séparer en mots
        unique_words = list(dict.fromkeys(words))  # Supprimer les doublons en conservant l'ordre
        return " ".join(unique_words)  # Recomposer la phrase

    # Créer un masque pour ne traiter que les lignes où 'merged' dépasse 1000 caractères
    mask = df_clean["merged"].astype(str).apply(len) > 1000

    # Appliquer la suppression des doublons uniquement aux lignes concernées
    df_clean.loc[mask, "merged"] = df_clean.loc[mask, "merged"].apply(remove_duplicate_words)

    # Sauvegarde du DataFrame mis à jour
    df_clean.to_csv("df_part1_sans_doublons_textes.csv", index=False)

    print(f"✅ Nouveau DataFrame enregistré : 'df_part1_sans_doublons_textes.csv'")
    print(f"📉 Suppression des doublons effectuée sur {mask.sum()} lignes ayant plus de 1000 caractères.")

else:
    print("❌ La colonne 'merged' n'existe pas dans df_part1_filtre_clean.csv")


✅ Nouveau DataFrame enregistré : 'df_part1_sans_doublons_textes.csv'
📉 Suppression des doublons effectuée sur 2824 lignes ayant plus de 1000 caractères.


Tronquer les textes plutôt que supprimer (1000 ou 750 caractéres)

In [5]:
import pandas as pd

# Charger le DataFrame filtré
df_filtered = pd.read_csv("df_part1_sans_doublons_textes.csv")

# Vérifier que la colonne "merged" existe
if "merged" in df_filtered.columns:
    # Calculer la longueur des textes
    df_filtered["text_length"] = df_filtered["merged"].astype(str).apply(len)

    # Calcul des statistiques
    max_length = df_filtered["text_length"].max()
    min_length = df_filtered["text_length"].min()
    median_length = df_filtered["text_length"].median()

    # Affichage des résultats
    print(f"📏 Longueur max : {max_length} caractères")
    print(f"📏 Longueur min : {min_length} caractères")
    print(f"📏 Médiane : {median_length} caractères")

else:
    print("❌ La colonne 'merged' n'existe pas dans df_part1_filtre.csv")

📏 Longueur max : 3247 caractères
📏 Longueur min : 3 caractères
📏 Médiane : 227.0 caractères


In [6]:
import pandas as pd

# Charger le DataFrame
df_clean = pd.read_csv("df_part1_sans_doublons_textes.csv")

# Vérifier que la colonne "merged" existe
if "merged" in df_clean.columns:
    # Fonction pour tronquer un texte en gardant un mot complet
    def truncate_text(text, max_length=750):
        text = str(text)  # S'assurer que c'est une chaîne de caractères
        if len(text) <= max_length:
            return text  # Retourner directement si pas besoin de tronquer
        
        # Tronquer jusqu'à la limite
        truncated = text[:max_length]
        
        # Trouver le dernier espace pour garder un mot complet
        last_space = truncated.rfind(" ")
        if last_space != -1:
            truncated = truncated[:last_space]  # Couper à l'espace trouvé
        
        return truncated

    # Appliquer le tronquage uniquement aux lignes dépassant 1000 caractères
    mask = df_clean["merged"].astype(str).apply(len) > 750
    df_clean.loc[mask, "merged"] = df_clean.loc[mask, "merged"].apply(truncate_text)

    # Sauvegarde du DataFrame mis à jour
    df_clean.to_csv("df_part1_tronque.csv", index=False)

    print(f"✅ Nouveau DataFrame enregistré : 'df_part1_tronque.csv'")
    print(f"📉 Tronquage effectué sur {mask.sum()} lignes ayant plus de 750 caractères.")

else:
    print("❌ La colonne 'merged' n'existe pas dans df_part1_sans_doublons_textes.csv")


✅ Nouveau DataFrame enregistré : 'df_part1_tronque.csv'
📉 Tronquage effectué sur 5064 lignes ayant plus de 750 caractères.


Repartition de langues pour Traduction

In [7]:
import pandas as pd
from langdetect import detect
from collections import Counter

# Charger le DataFrame
df2 = pd.read_csv("df_part1_tronque.csv", index_col=0)

# Vérifier que la colonne "merged_trad" existe
if "merged" in df2.columns:
    # Fonction de détection de la langue avec gestion des erreurs
    def detect_lang(text):
        try:
            return detect(str(text))  # Détection de la langue
        except:
            return "unknown"  # Si erreur, on met "unknown"

    # Ajouter une colonne "lang" avec la langue détectée
    df2["lang"] = df2["merged"].apply(detect_lang)

    # Filtrer les textes dont la langue détectée n'est pas le français
    df_non_fr = df2[df2["lang"] != "fr"]

    # Calculer le nombre total de caractères dans "merged_trad" pour les langues non françaises
    total_chars_non_fr = df_non_fr["merged"].astype(str).apply(len).sum()

    # Calculer la répartition des langues détectées
    lang_counts = Counter(df2["lang"])
    lang_df = pd.DataFrame(lang_counts.items(), columns=["Langue", "Nombre de textes"])
    lang_df["Pourcentage"] = (lang_df["Nombre de textes"] / len(df2)) * 100

    # Affichage des résultats
    print(f"\n✅ Nombre total de caractères dans 'merged' (langues ≠ fr) : {total_chars_non_fr}\n")
    print("📊 Répartition des langues détectées :")
    print(lang_df.sort_values(by="Nombre de textes", ascending=False).to_string(index=False))

else:
    print("❌ La colonne 'merged' n'existe pas dans df_part1_tronque.csv")



✅ Nombre total de caractères dans 'merged' (langues ≠ fr) : 947665

📊 Répartition des langues détectées :
Langue  Nombre de textes  Pourcentage
    fr             19485    73.084280
    en              3994    14.980683
    it               805     3.019392
    ca               333     1.249015
    ro               326     1.222760
    es               191     0.716402
    no               157     0.588875
    da               154     0.577623
    nl               149     0.558869
    af               115     0.431342
    de               114     0.427591
    pt               101     0.378831
    sv                93     0.348824
    id                78     0.292562
    hr                78     0.292562
    cy                71     0.266307
    sl                63     0.236300
    et                56     0.210045
    tl                54     0.202543
    fi                51     0.191291
    sk                30     0.112524
    pl                29     0.108773
    sq             

In [8]:
import pandas as pd
from langdetect import detect

# Charger le DataFrame
df2 = pd.read_csv("df_part1_tronque.csv", index_col=0)

# Vérifier que la colonne "merged" existe
if "merged" in df2.columns:
    # Fonction de détection de la langue avec gestion des erreurs
    def detect_lang(text):
        try:
            return detect(str(text))  # Détection de la langue
        except:
            return "unknown"  # Si erreur, on met "unknown"

    # Ajouter une colonne "lang" avec la langue détectée
    df2["lang"] = df2["merged"].apply(detect_lang)

    # Répartition des langues détectées
    lang_counts = df2["lang"].value_counts(normalize=True)  # Proportion
    filtered_langs = lang_counts[lang_counts >= 0.025].index  # Garde langues ≥ 2.5%

    # Filtrer le DataFrame
    df_filtered = df2[df2["lang"].isin(filtered_langs)]

    # Calculer le nombre total de caractères dans "merged" pour les langues non françaises
    df_non_fr = df_filtered[df_filtered["lang"] != "fr"]
    total_chars_non_fr = df_non_fr["merged"].astype(str).apply(len).sum()

    # Afficher la nouvelle répartition des langues après filtrage
    lang_counts_filtered = df_filtered["lang"].value_counts().reset_index()
    lang_counts_filtered.columns = ["Langue", "Nombre de lignes"]
    lang_counts_filtered["Pourcentage"] = (lang_counts_filtered["Nombre de lignes"] / len(df_filtered)) * 100

    # Sauvegarder le DataFrame filtré
    df_filtered.to_csv("df_part1_filtre.csv", index=False)

    # Affichage des résultats
    print(f"\n✅ Nombre total de caractères dans 'merged' (langues ≠ fr) : {total_chars_non_fr}\n")
    print("📊 Répartition des langues détectées (après suppression des langues < 2.5%) :")
    print(lang_counts_filtered.to_string(index=False))

else:
    print("❌ La colonne 'merged' n'existe pas dans df_part1.csv")



✅ Nombre total de caractères dans 'merged' (langues ≠ fr) : 844973

📊 Répartition des langues détectées (après suppression des langues < 2.5%) :
Langue  Nombre de lignes  Pourcentage
    fr             19461    80.099605
    en              4001    16.467731
    it               834     3.432664


. . regarder le nb de caractére a traduire (hors Fr) puis supprimer les lignes de langues >3% . 

In [9]:
import pandas as pd

# Charger le DataFrame filtré
df_filtered = pd.read_csv("df_part1_filtre.csv")

# Vérifier que la colonne "merged" existe
if "merged" in df_filtered.columns:
    # Calculer la longueur des textes
    df_filtered["text_length"] = df_filtered["merged"].astype(str).apply(len)

    # Calcul des statistiques
    max_length = df_filtered["text_length"].max()
    min_length = df_filtered["text_length"].min()
    median_length = df_filtered["text_length"].median()

    # Affichage des résultats
    print(f"📏 Longueur max : {max_length} caractères")
    print(f"📏 Longueur min : {min_length} caractères")
    print(f"📏 Médiane : {median_length} caractères")

else:
    print("❌ La colonne 'merged' n'existe pas dans df_part1_filtre.csv")

📏 Longueur max : 750 caractères
📏 Longueur min : 4 caractères
📏 Médiane : 282.0 caractères


In [10]:
import pandas as pd

# Charger le DataFrame filtré
df_filtered = pd.read_csv("df_part1_filtre.csv")

# Vérifier que la colonne "merged" existe
if "merged" in df_filtered.columns:
    # Calculer la longueur des textes
    df_filtered["text_length"] = df_filtered["merged"].astype(str).apply(len)

    # Filtrer les lignes dépassant 500 caractères
    df_too_long = df_filtered[df_filtered["text_length"] > 1000]
    df_filtered = df_filtered[df_filtered["text_length"] <= 1000]

    # Nombre de lignes supprimées
    num_removed = len(df_too_long)
    print(f"🗑️ Nombre de lignes supprimées : {num_removed}")

    # Supprimer la colonne temporaire 'text_length'
    df_filtered = df_filtered.drop(columns=["text_length"])

    # Sauvegarde du DataFrame nettoyé
    df_filtered.to_csv("df_part1_filtre_clean.csv", index=False)
    print("✅ Fichier nettoyé sauvegardé sous 'df_part1_filtre_clean.csv'")

else:
    print("❌ La colonne 'merged' n'existe pas dans df_part1_filtre.csv")

🗑️ Nombre de lignes supprimées : 0
✅ Fichier nettoyé sauvegardé sous 'df_part1_filtre_clean.csv'


In [11]:
import pandas as pd
from langdetect import detect

# Charger le DataFrame
df2 = pd.read_csv("df_part1_filtre_clean.csv", index_col=0)

# Vérifier que la colonne "merged" existe
if "merged" in df2.columns:
    # Fonction de détection de la langue avec gestion des erreurs
    def detect_lang(text):
        try:
            return detect(str(text))  # Détection de la langue
        except:
            return "unknown"  # Si erreur, on met "unknown"

    # Ajouter une colonne "lang" avec la langue détectée
    df2["lang"] = df2["merged"].apply(detect_lang)

    # Répartition des langues détectées
    lang_counts = df2["lang"].value_counts(normalize=True)  # Proportion
    filtered_langs = lang_counts[lang_counts >= 0.03].index  # Garde langues ≥ 3%

    # Filtrer le DataFrame
    df_filtered = df2[df2["lang"].isin(filtered_langs)]

    # Calculer le nombre total de caractères dans "merged" pour les langues non françaises
    df_non_fr = df_filtered[df_filtered["lang"] != "fr"]
    total_chars_non_fr = df_non_fr["merged"].astype(str).apply(len).sum()

    # Afficher la nouvelle répartition des langues après filtrage
    lang_counts_filtered = df_filtered["lang"].value_counts().reset_index()
    lang_counts_filtered.columns = ["Langue", "Nombre de lignes"]
    lang_counts_filtered["Pourcentage"] = (lang_counts_filtered["Nombre de lignes"] / len(df_filtered)) * 100

    # Affichage des résultats
    print(f"\n✅ Nombre total de caractères dans 'merged' (langues ≠ fr) : {total_chars_non_fr}\n")
    print("📊 Répartition des langues détectées (après suppression des langues < 3%) :")
    print(lang_counts_filtered.to_string(index=False))

else:
    print("❌ La colonne 'merged' n'existe pas dans df_part1.csv")


✅ Nombre total de caractères dans 'merged' (langues ≠ fr) : 833492

📊 Répartition des langues détectées (après suppression des langues < 3%) :
Langue  Nombre de lignes  Pourcentage
    fr             19374    80.527038
    en              3915    16.272497
    it               770     3.200466


In [12]:
import pandas as pd
from langdetect import detect

# Charger le DataFrame
df2 = pd.read_csv("df_part1_filtre_clean.csv", index_col=0)

# Vérifier que la colonne "merged" existe
if "merged" in df2.columns:
    # Fonction de détection de la langue avec gestion des erreurs
    def detect_lang(text):
        try:
            return detect(str(text))  # Détection de la langue
        except:
            return "unknown"  # Si erreur, on met "unknown"

    # Ajouter une colonne "lang" avec la langue détectée
    df2["lang"] = df2["merged"].apply(detect_lang)

    # Calculer la répartition des langues détectées (en nombre de lignes)
    lang_counts = df2["lang"].value_counts()
    total_rows = len(df2)
    
    # Supprimer les langues représentant moins de 3% des lignes
    lang_threshold = total_rows * 0.03
    lang_to_remove = lang_counts[lang_counts < lang_threshold].index
    df2_filtered = df2[~df2["lang"].isin(lang_to_remove)]

    # Calculer le nombre total de caractères non français restant
    df_non_fr = df2_filtered[df2_filtered["lang"] != "fr"]
    total_chars_non_fr = df_non_fr["merged"].astype(str).apply(len).sum()

    print(f"Nombre total de caractères dans 'merged' (langues ≠ fr) après suppression : {total_chars_non_fr}")

    # Sauvegarder le DataFrame filtré dans un nouveau fichier CSV
    df2_filtered.to_csv("df_part1_traduit_filtered.csv", index=False)

else:
    print("❌ La colonne 'merged' n'existe pas dans df_part1_filtre_clean.csv")



Nombre total de caractères dans 'merged' (langues ≠ fr) après suppression : 826806


In [13]:
import pandas as pd
import deepl

DEEPL_API_KEY = "54778c8f-41cf-48b3-ad8d-96b12a2f1260"

# Charger le DataFrame
df = pd.read_csv("df_part1_traduit_filtered.csv", index_col=0)

# Vérifier que la colonne "merged" existe
if "merged" in df.columns:
    # Initialiser le traducteur DeepL
    translator = deepl.Translator(DEEPL_API_KEY)

    # Fonction de traduction
    def translate_text(text, target_language="FR"):
        text = str(text)[:4000]  # Tronquer les textes > 4000 caractères (limite DeepL)
        try:
            result = translator.translate_text(text, target_lang=target_language)
            return result.text
        except Exception as e:
            print(f"Erreur lors de la traduction : {e}")
            return text  # Garde le texte original en cas d'erreur

    # Calcul du total de caractères traduits
    char_count = 0
    translated_texts = []
    
    for text in df["merged"]:
        if char_count >= 900_000:  # Stop si on dépasse 900 000 caractères
            translated_texts.append(text)  # On garde le texte original
        else:
            translated = translate_text(text)
            translated_texts.append(translated)
            char_count += len(text)  # Ajouter la longueur du texte traduit

    # Ajouter la colonne traduite au DataFrame
    df["merged_trad"] = translated_texts

    # Sauvegarder le DataFrame avec les traductions partielles
    df.to_csv("df_part1_traduit.csv", index=True)

    print(f"✅ Traduction terminée (800 000 caractères traduits) et fichier sauvegardé : df_part1_traduit.csv")
else:
    print("❌ La colonne 'merged' n'existe pas dans df_part1.csv")


✅ Traduction terminée (800 000 caractères traduits) et fichier sauvegardé : df_part1_traduit.csv
